# 1. 데이터 다운로드
/aiffel/lyricist/data/lyrics/*

# 2. 데이터 읽어오기

In [1]:
import os
import re
import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [3]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담음
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))

데이터 크기: 187088


# 3. 데이터 다듬기

In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()  # 소문자, 양쪽공백 제거
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)  # 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    sentence = sentence.strip()  # 양쪽 공백 제거
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

In [5]:
corpus = []  # 형태 : ['<start> i m begging of you please don t take my man <end>', ...] length - 175986
for sentence in raw_corpus:
    if len(sentence) == 0: continue
    tmp = preprocess_sentence(sentence)
    if len(tmp.split()) > 15: continue
    corpus.append(tmp)

In [6]:
def tokenize(corpus):
    # num_words:전체 단어의 개수, filters:별도로 전처리 로직을 추가, oov_token: out-of-vocabulary 사전에 없었던 단어는 어떤 토큰으로 대체할지
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=12000, filters=' ', oov_token="<unk>")
    tokenizer.fit_on_texts(corpus)  # corpus로부터 Tokenizer가 사전을 자동구축

    # tokenizer를 활용하여 모델에 입력할 데이터셋 구축(Tensor로 변환)
    tensor = tokenizer.texts_to_sequences(corpus)

    # 입력 데이터 시퀀스 길이 맞춰주기 - padding
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)

    return tensor, tokenizer


tensor, tokenizer = tokenize(corpus)

In [7]:
# 단어 사전이 어떻게 구축되었는지 확인 방법
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [8]:
# 3. 평가데이터셋 분리
"""
x_train : 소스 문장, 형식(<start> 문장), 즉 <end>를 삭제
y_train : 타겟 문장, 형식(문장 <end>), 즉 <start>를 삭제
단어장의 크기는 12,000 이상으로 설정하세요! 총 데이터의 20%를 평가 데이터셋으로 사용해 주세요!
"""
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높다.
tgt_input = tensor[:, 1:]  # tensor에서 <start>를 잘라내서 타겟 문장을 생성 -> 문장 길이는 14가 됨

# train data를 train, valid로 나눈다.(비율 80:20) 만약 학습데이터 개수가 124960보다 크다면 위 Step 3.의 데이터 정제 과정을 다시 검토
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=20)
print("Source Train:", enc_train.shape)  # (124960, 14)  # 현재 (124981, 14)
print("Target Train:", dec_train.shape)  # (124960, 14)

Source Train: (124981, 14)
Target Train: (124981, 14)


# 4. 모델 생성

In [9]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)  # 입력된 텐서에는 단어사전의 인덱스가 들어있는데, 이 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)

        return out


embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

# 5. 모델 학습

In [10]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10, validation_data=(enc_val, dec_val))


def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]  # 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됨

        # 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여줌
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

Epoch 1/10
3906/3906 [==============================] - 147s 36ms/step - loss: 3.0829 - val_loss: 2.8246
Epoch 2/10
3906/3906 [==============================] - 141s 36ms/step - loss: 2.6598 - val_loss: 2.6250
Epoch 3/10
3906/3906 [==============================] - 141s 36ms/step - loss: 2.3823 - val_loss: 2.5020
Epoch 4/10
3906/3906 [==============================] - 141s 36ms/step - loss: 2.1297 - val_loss: 2.4289
Epoch 5/10
3906/3906 [==============================] - 141s 36ms/step - loss: 1.9076 - val_loss: 2.3836
Epoch 6/10
3906/3906 [==============================] - 141s 36ms/step - loss: 1.7175 - val_loss: 2.3594
Epoch 7/10
3906/3906 [==============================] - 141s 36ms/step - loss: 1.5575 - val_loss: 2.3553
Epoch 8/10
3906/3906 [==============================] - 141s 36ms/step - loss: 1.4241 - val_loss: 2.3646
Epoch 9/10
3906/3906 [==============================] - 141s 36ms/step - loss: 1.3159 - val_loss: 2.3915
Epoch 10/10
3906/3906 [==============================] 

In [11]:
test_sen = generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)
print(test_sen)

<start> i love you <end> 


# [E-04] 회고

데이터를 다운 받고, 읽고, 다듬고, 모델 생성 후 학습까지는 좋았습니다. 그래서 이번 exploration은 LMS노드와 함께 구글링을 하며 해낼 줄 알았습니다😂

그런데 진짜 마지막에... 골인 지점을 눈앞에 놔두고 'i love'가 들어간 문장을 출력하는 부분에서 뭔가가 삐걱거리기 시작했는데, 그 뭔가를 알 수 없어서 구글링을 했습니다.

자료를 찾아보면서 코드를 바꿔보고 실행해보고 괜찮아서 넘어갔다 싶으면 그 다음이 또 문제고😅 한 발짝 두 발짝 다가가면 세 발짝 멀어져서 어디가 문제인지, 어디쯤에 있는지 몰라서 다시 처음부터 진행했습니다.


(bgm🎵 한 발짝 두 발짝 🎤오마이걸)

다시 처음 부터 시작하면서 자신있게 데이터 정제 부분까지 마치고, 모델을 생성하는 순간부터 작년에 해당 exploration을 진행하셨던 분의 블로그를 참고(라고 쓰고 ctrl+c, ctrl+v)해 exploration이 완성될 수 있도록 만들었습니다.

https://velog.io/@nameunzz/%EC%9E%91%EC%82%AC%EA%B0%80-%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5-%EB%A7%8C%EB%93%A4%EA%B8%B0

가사 텍스트 생성 모델이 정상적으로 작동하고 있고 텍스트 제너레이션 결과가 그럴듯한 문장으로 생성 되었는 것 같은데, 왜 문장이 하나 밖에 출력이 안 되었을까요?

이건 제가 처음에 데이터를 다운받을 때 클라우드 쉘에서 압축을 푸는 과정에서 제가 듣기 싫은 가수는 압축해제를 안 해서 나온 결과 같습니다😅

그리고 텍스트 생성모델의 validation loss가 2.2 이하로 낮아지지는 않았습니다. 에포크를 10할당하고 돌렸을때 처음에는 LMS에 있던 그대로 embedding_size = 256, hidden_size = 1024를 주었을 때 2.2이상으로 나와서 일꾼을 3000까지 늘렸는데 최소 2.3Rkwl sofurkrh 에포크 7번째를 넘어가면 다시 validation loss 값이 올라갔습니다. 그래서 결국 다시 일꾼을 줄이고 제자리로 돌아왔습니다😭

validation loss 줄이는 방법을 검색해봤는데 데이터 전처리, 모델의 강제성(복잡하지 않는지 등), 학습속도 줄이기 등이 있었습니다.
exploration 4를 처음 부터 다시 할 때 최대한 불피요 해보이는 것들(배치 확인 등)을 제외하고 다시 만들어서 저는 복잡하지 않다고 생각했는데... 프로그램은 복잡하다고 생각하는지 validation loss 값이 줄어들지 않았습니다.

이번 노드를 실행하고 학습자료를 찾아보면서 제가 프로그램을 어떻게 만들고 있는지, 잘 이해하면서 만들고 있는지 다시 생각해보는 시간이었던 것 같습니다.
그래도 아직 잘 모르니 다양한 문제들을 계속 보고, 심심하면 보고, 할 거 없을때 보고 그냥 많이 봐두고 찾아보는게 답인 것 같습니다.